In [1]:
import netCDF4
import pandas as pd
import geopandas as gpd
import numpy as np
import os
from glob import glob
from osgeo import gdal

### The below class is initiated with a direcory containing netCDF files. The class constructs itself by creating a dataframe and geodataframe object. The class contains a method to filter the original dataframe given a QA Value threshold. For example, if 0.75 is passed, only QA values greater than 0.75 will be returned.

In [2]:
class netCDF():
    def __init__(self, directory):
        dfList = []
        currentDirectory = os.chdir(directory)
        netCDFFiles = glob("*.nc")
        for file in netCDFFiles:
            data = netCDF4.Dataset(os.path.join(directory, file))
            product = data.groups['PRODUCT']
            latDF = pd.DataFrame(np.array(product.variables['latitude']).flatten(), columns=['latitude'])
            lonDF = pd.DataFrame(np.array(product.variables['longitude']).flatten(), columns=['longitude'])
            N02TropoDF = pd.DataFrame(np.array(product.variables['nitrogendioxide_tropospheric_column']).flatten(), columns = ['N02'])
            qaValuesDF = pd.DataFrame(np.array(product.variables['qa_value']).flatten(), columns = ['QAValue'])
            df = latDF.merge(lonDF, left_index=True, right_index=True)
            df = df.merge(N02TropoDF, left_index=True, right_index=True)
            df = df.merge(qaValuesDF, left_index=True, right_index=True)
            dfList.append(df)
        self.df = pd.concat(dfList)
        self.geoDF = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))
        
    def filterQAValue(self,value):
        filteredDF = self.df.query('QAValue > ' + str(value))
        return filteredDF
        
    

In [3]:
path = r"/Users/kevinboyle/Documents/SentinelData"

In [4]:
test = netCDF(path)

In [8]:
len(test.df)

334800

In [5]:
test.geoDF.head()

,latitude,longitude,N02,QAValue,geometry
0,32.364525,117.091576,9.969210e+36,0.00,POINT (117.09158 32.36452)
1,32.407211,117.178032,2.933390e-04,0.33,POINT (117.17803 32.40721)
2,32.449253,117.263390,1.517603e-04,0.33,POINT (117.26339 32.44925)
3,32.490669,117.347672,-8.594624e-05,0.33,POINT (117.34767 32.49067)
4,32.531471,117.430908,-1.847283e-04,0.33,POINT (117.43091 32.53147)


In [6]:
test.filterQAValue(.75)

,latitude,longitude,N02,QAValue
48,34.458431,121.643723,0.000033,1.00
49,34.495884,121.732651,0.000031,1.00
50,34.532722,121.820480,0.000032,1.00
51,34.568966,121.907234,0.000032,1.00
52,34.604630,121.992943,0.000030,1.00
...,...,...,...,...
167330,56.355881,134.480988,0.000003,0.88
167331,56.357723,134.595261,0.000004,0.88
167332,56.359436,134.710739,0.000005,0.88
167333,56.361015,134.827438,0.000003,0.88
